In [191]:
import requests
import json
from bs4 import BeautifulSoup as bs
from api.iterating_api import generator_over_review
from api.parsers_api import headers, HTTP_error, parse_GameSpot

In [49]:


url_404 = "http://www.gamespot.com/pc/strategy/dominions3/review.html?sid=6163352"
data_path = "/Users/fedor/Desktop/meta_project/data/metacritic/critic_review"
logs_path = "/Users/fedor/Desktop/meta_project/data/metacritic/logs/logs_1.txt"
save_path = "/Users/fedor/Desktop/meta_project/data/metacritic/outter_critic_review/"


# SpotGame reviews by links from metacritic

In [52]:
def my_predicate(d):
    return d["name"] == "GameSpot"


a = generator_over_review(data_path,
                          # game_search_list=['INSIDE'],
                          # platform_search_list=["pc"],
                          review_predicate = my_predicate,
                          add_info=False)
COUNTER = 0

print("Do you want to delete pld logs? Enter + if so")
if input() == "+":
    open(logs_path, 'w').close() #clean up logs file
    print("-log file cleaned up")

with open(logs_path, "a") as log_file:

    for rev in a:
        COUNTER += 1
        if COUNTER < 2036:
            continue
        if not rev['ref']:
            continue
        try:
            parse_GameSpot(rev['ref'],save_path = save_path + str(COUNTER)+".json", json_save = True)

        except HTTP_error as e:

            if e.status_code == 404:
                log_file.write("404 "+e.link + "\n")
            elif e.status_code == 0:
                log_file.write("000 " + e.link + "\n")
            else:
                log_file.write(str(e.status_code) + " " + e.link + "\n")

        except requests.ConnectionError or requests.HTTPError or requests.exceptions.RequestException or requests.exceptions.MissingSchema:
            # print("Inner error of Requests lib", rev['ref'])
            log_file.write("INN " + rev['ref'] + "\n")

        except ... :
            log_file.write("VBL " + rev['ref'] + "\n")
            # print("Very broken link", rev['ref'])

Do you want to delete pld logs? Enter + if so


# Parsing SpotGame reviews by links from SpotGame itself.
Links come from structure of a web-site structure.

In [193]:
def parse_GameSpot(URL: str,
                   save_path: str = "",
                   json_save: bool = False
                   ):
    """
    Gets information about review about game on a page (name of game,
    headline of article, text of review) and returns it as dict. Works only with that site!
    IGN main page - https://www.gamespot.com
    :param URL: str
        page url
    :param  save_path: str
        Path to save data on json format
    :return: None or dict
        if dict - full of info (keys: name_review, ref, text)
    """

    # get HTTP page by GET request
    r = requests.get(URL, headers=headers, timeout=10)

    # HTTP check (f.e. for 404 request status)
    if not r.ok:
        raise HTTP_error(r.status_code, r.url, "Big error")

    # HTML code scrubbing start
    soup = bs(r.text, "html.parser")
    body = soup.find(class_="js-content-entity-body")

    if not body:
        redirect_ref = soup.find(class_="media media-game media-game")
        if not redirect_ref:
            er = HTTP_error(0, r.url, "Wierd page - can not scrab!")
            raise er

        return parse_GameSpot("https://www.gamespot.com" + redirect_ref.a['href'], save_path, json_save)

    p_text_list = body.find_all("p")

    game_full_rewiev = ""
    for p in p_text_list:
        game_full_rewiev += p.text

    # game_header = soup.find(class_="news-title instapaper_title entry-title").text
    date = soup.find(class_="news-byline").time.text

    date = date[:date.find("at")]  #they provide in format February 23, 2005 at 1:41PM PST

    game_name = soup.find(class_="no-hover").span.text
    # HTML code scrubbing end

    # Dictionary with data forming
    data = {
        # 'name_review': game_header,
        'ref': URL,
        'date': date,
        'game_name': game_name,
        'text': game_full_rewiev}

    # Save dictionary as json or return
    if json_save:
        with open(save_path, 'w') as f:
            json.dump(data, f)
        return
    else:
        return data


In [56]:
from tqdm import tqdm

for url in tqdm(url_list[143+156+213+19+67+41:]):

    r = requests.get(url, headers=headers, timeout=10)
    soup = bs(r.text, "html.parser")
    body = soup.find(id ="js-sort-filter-results").find("section")
    for segment in body.find_all("a"):
        absolut_url.append(spot_prefics+segment["href"])


100%|██████████| 95/95 [03:04<00:00,  1.94s/it]


In [196]:
curent_position = 9
save_path = "/Users/fedor/Desktop/meta_project/data/metacritic/gamespot/"
with open("/Users/fedor/Desktop/meta_project/data/metacritic/logs/logs_2.txt", "a") as log_file:
    for COUNTER, url in enumerate(tqdm(absolut_url_new[curent_position:])):
        try:
            parse_GameSpot(url, save_path=save_path + str(COUNTER) + ".json", json_save=True)

        except HTTP_error as e:
            if e.status == 404:
                log_file.write("404 " + url + "\n")
            elif e.status == 0:
                log_file.write("000 " + url + "\n")
            else:
                log_file.write(str(e.status) + " " + url + "\n")

        except requests.ConnectionError or requests.HTTPError or requests.exceptions.RequestException or requests.exceptions.MissingSchema:
            log_file.write("INN " + url + "\n")

        except requests.exceptions.Timeout:
            try:
                parse_GameSpot(url, save_path=save_path + str(COUNTER) + ".json", json_save=True)
            except requests.exceptions.Timeout:
                log_file.write("TOU " + url + "\n")

        except ...:
            log_file.write("VBL " + url + "\n")

100%|██████████| 15380/15380 [4:25:35<00:00,  1.04s/it]  


# Error Analysis
Total amount of urls if 15.380

In [200]:
er_dict = {}
print()
with open("/Users/fedor/Desktop/meta_project/data/metacritic/logs/logs_2.txt", "r") as log_file:
    for line in log_file.readlines():
        er_code = line[:3]
        if er_code not in er_dict.keys():
            er_dict[er_code] = 1
        else:
            er_dict[er_code] += 1
er_dict

{'\n': 1, '000': 25, '502': 20, '404': 1}

# Result analyses
and data reforming

In [248]:
from pathlib import Path
folder_path = "/Users/fedor/Desktop/meta_project/data/metacritic/game_spot/spotgame_links"
alternative_folder_path = "/Users/fedor/Desktop/meta_project/data/metacritic/game_spot/spotgame_links_1/"
name_dict = {}
names = Path(folder_path).glob('*')
wrong_names = []
COUNTER = 0
for n in tqdm(names):
    COUNTER +=  1
    if COUNTER == 2311:
        continue

    with open(n,"r") as old_file:
        template = json.load(old_file)
        game_name = template["game_name"]

        if  game_name.find("/"):
            wrong_names.append(game_name)
            game_name = game_name.replace("/","_")

        if game_name in name_dict:
            name_dict[game_name] += 1
        else:
            name_dict[game_name] = 1

        with open(alternative_folder_path+game_name+"_"+str(name_dict[game_name])+".json","w")as f:
            json.dump(template, f)




15335it [00:23, 654.52it/s]


In [255]:
s = sorted(name_dict, key= lambda x: name_dict[x],reverse=True)

In [264]:
for l in s[:400]:
    print(name_dict[l], l)

17 The Walking Dead: A Telltale Games Series
12 Fallout: New Vegas
11 The Sims
11 The Sims 3
11 Mass Effect 2
10 Need for Speed: Carbon
10 Madden NFL 07
9 Madden NFL 06
9 Disney_Pixar Ratatouille
9 Surf's Up
9 Madden NFL 08
9 World of Warcraft
9 Need for Speed: Most Wanted (2005)
9 Open Season
8 Sam & Max: Save the World
8 TMNT
8 Disney_Pixar Cars
8 Marvel: Ultimate Alliance
8 Star Wars: The Force Unleashed
8 Harry Potter and the Order of the Phoenix
8 Lego Star Wars II: The Original Trilogy
8 Spider-Man 2
8 FIFA World Cup: Germany 2006
8 FIFA 07 Soccer
8 Diablo III
8 Tom Clancy's Splinter Cell: Chaos Theory
8 Borderlands
8 Dishonored
8 Tomb Raider: Legend
8 Dragon Age: Origins
7 Eragon
7 Lego Indiana Jones: The Original Adventures
7 FIFA Soccer 2005
7 Tiger Woods PGA Tour 07
7 Tom Clancy's Splinter Cell
7 Prince of Persia: The Two Thrones
7 Disney Pirates of the Caribbean: At World's End
7 Disney's Meet the Robinsons
7 Major League Baseball 2K7
7 Harry Potter and the Goblet of Fire
7 

In [270]:
data_path = "/Users/fedor/Desktop/meta_project/data/metacritic/metacritic/critic_review"
meta_names = Path(data_path).glob('*')
g_name_list = []
for g_name in tqdm(meta_names):
    g_name_list.append((str(g_name).split("/")[-1]).replace("/","_"))

12899it [00:00, 60205.61it/s]


In [275]:
sum = 0
for spot_name in name_dict:
    if spot_name in g_name_list:
        sum += name_dict[spot_name]
sum # total amount of suitable data

10957

In [278]:
import os
f_path = "/Users/fedor/Desktop/meta_project/data/metacritic/game_spot/spotgame_final"
old_path = "/Users/fedor/Desktop/meta_project/data/metacritic/game_spot/spotgame_links"
COUNTER = 0
t_name_dict = {}
names = Path(old_path).glob('*')
wrong_names = []
for n in tqdm(names):
    COUNTER +=  1
    if COUNTER == 2311:
        continue
    with open(n,"r") as old_file:
        template = json.load(old_file)
        game_name = template["game_name"].replace("/","_")
        if game_name not in g_name_list:
            continue

        if game_name in t_name_dict:
            t_name_dict[game_name] += 1
        else:
            t_name_dict[game_name] = 1

        if game_name in g_name_list:
            if not os.path.exists(f_path+"/"+game_name):
                os.mkdir(f_path+"/"+game_name)

            with open(f_path+"/"+game_name+"/"+str(t_name_dict[game_name])+".json","w")as f:
                json.dump(template, f)


15335it [00:29, 517.61it/s]
